In [1]:
%matplotlib inline  

Load synthetic data

In [2]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
data = load_breast_cancer()
x = MinMaxScaler().fit_transform(data.data)
y = data.target
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=0)

Train PLNN

In [3]:
import keras

def build_plnn(input_dim, hidden_layer_sizes):

    hidden_layer_sizes = [input_dim] + hidden_layer_sizes
    model = keras.models.Sequential()
    for i in range(len(hidden_layer_sizes) - 1):
        model.add(keras.layers.Dense(
            input_dim=hidden_layer_sizes[i],
            units=hidden_layer_sizes[i + 1],
            kernel_initializer='he_uniform',
            bias_initializer='zeros',
            activation='relu') 
        )
        
    model.add(keras.layers.Dense(
            units=1,
            input_dim=hidden_layer_sizes[-1],
            kernel_initializer='glorot_uniform',
            bias_initializer='zeros',
            activation='sigmoid') 
        )

    # compile model
    model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
                    loss='binary_crossentropy')
    return model

mlp = build_plnn(train_x.shape[1], [20] * 2)
mlp.fit(train_x, train_y, epochs=2000, batch_size=200, verbose=False)

Using TensorFlow backend.


Use OpenBox to interpret the fitted PLNN

In [5]:
from aletheia import *

coefs = [layer.kernel.numpy() for layer in mlp.layers]
intercepts = [layer.bias.numpy() for layer in mlp.layers]
clf = UnwrapperClassifier(coefs, intercepts)
clf.fit(x, y)

,Count,Response Mean,Response Std,Local AUC,Global AUC
0,50.0,1.0,0.0,NaN,0.984977
1,35.0,1.0,0.0,NaN,0.987593
2,28.0,1.0,0.0,NaN,0.991755
3,27.0,0.0,0.0,NaN,0.989786
4,16.0,0.0,0.0,NaN,0.989073
...,...,...,...,...,...
103,1.0,0.0,0.0,NaN,0.995759
102,1.0,0.0,0.0,NaN,0.990870
101,1.0,1.0,0.0,NaN,0.990051
100,1.0,1.0,0.0,NaN,0.983709
